In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[1]") \
.appName("SparkByExamples.com") \
.config('job.local.dir', 'file:/Users/parvez.mullah/Documents/LearningTechnologies/Big Data Tech/Spark/data') \
.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/07/28 15:12:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Remove double quotes from value of json string

In [31]:
from pyspark.sql.functions import split, col, lit, concat_ws, concat

json_string = '{"id":"2","name":"Parvez"Mullah","city":"mumbai"}'

data = [(1, json_string)]

cols = ["col1", "col2"]

df = spark.createDataFrame(data=data, schema=cols)

df = df.withColumn("col3", split(col("col2"), '"name":"')[0])\
.withColumn("col4", lit('"name":"'))\
.withColumn("col5", split(col("col2"), '"name":"')[1])

df = df.withColumn("col6", split(col("col5"), '"', 2))
df = df.withColumn("col7", concat_ws(' ', col("col6")))
df = df.withColumn("col8", concat(col("col3"), col("col4"), col("col7")))
df.show(truncate=False)

+----+-------------------------------------------------+----------+--------+-------------------------------+----------------------------------+-------------------------------+-------------------------------------------------+
|col1|col2                                             |col3      |col4    |col5                           |col6                              |col7                           |col8                                             |
+----+-------------------------------------------------+----------+--------+-------------------------------+----------------------------------+-------------------------------+-------------------------------------------------+
|1   |{"id":"2","name":"Parvez"Mullah","city":"mumbai"}|{"id":"2",|"name":"|Parvez"Mullah","city":"mumbai"}|[Parvez, Mullah","city":"mumbai"}]|Parvez Mullah","city":"mumbai"}|{"id":"2","name":"Parvez Mullah","city":"mumbai"}|
+----+-------------------------------------------------+----------+--------+--------------------

# Check if a column exisits in DataFrame

In [41]:
data = [(1, 'Parvez', 'Male', 2000), (2, 'Parry', 'Male', 3000)]

columns = ['id', 'name', 'gender', 'salary']

df = spark.createDataFrame(data, columns)

field_names = df.schema.fieldNames()

'id' in field_names

'absent_field' in field_names

False

# Different Ways to apply functions on column in DataFrame

In [43]:
data = [(1, 'Parvez', 'Male', 2000), (2, 'Parry', 'Male', 3000)]

columns = ['id', 'name', 'gender', 'salary']

df = spark.createDataFrame(data, columns)

In [44]:
from pyspark.sql.functions import upper

df.withColumn("upper_name", upper(df.name)).show()

+---+------+------+------+----------+
| id|  name|gender|salary|upper_name|
+---+------+------+------+----------+
|  1|Parvez|  Male|  2000|    PARVEZ|
|  2| Parry|  Male|  3000|     PARRY|
+---+------+------+------+----------+



In [47]:
df.select('id', upper('name').alias('upper_name')).show()

+---+----------+
| id|upper_name|
+---+----------+
|  1|    PARVEZ|
|  2|     PARRY|
+---+----------+



In [50]:
df.createOrReplaceTempView('persons')

spark.sql('Select id, UPPER(name) as upper_name from persons').show()

+---+----------+
| id|upper_name|
+---+----------+
|  1|    PARVEZ|
|  2|     PARRY|
+---+----------+



In [52]:
def upper_name(df):
    return df.withColumn('name', upper(df.name))

df.transform(upper_name).show()

+---+------+------+------+
| id|  name|gender|salary|
+---+------+------+------+
|  1|PARVEZ|  Male|  2000|
|  2| PARRY|  Male|  3000|
+---+------+------+------+

